# Parallel processing
Er zijn veel verschillende packages die parallel processing mogelijk maken. In dit voorbeeld maken we gebruik van de package [joblib](https://joblib.readthedocs.io/en/latest/) waarmee vrij gemakkelijk een functie op meer dan een process kan worden berekend.

Als `joblib` nog niet is geinstalleerd in jouw environment, ga dan naar de anaconda prompt, en gebruik het volgende installatie commando:

`$ conda install -c conda-forge joblib`

In [ ]:
from joblib import Parallel, delayed
import time

# Opsplitsen in Functies
Bedenk welke taak je parallel wil laten uitvoeren om de taak die je wilt doen te versnellen. Een vereiste om verder te kunnen is dat deze taak door een functie kan worden uitgevoerd. Sommige operaties kunnen niet tegelijk, zoals in twee processen hetzelfde bestand overschrijven. Zolang je gescheiden in- en output hebt, kan het parallel draaien.

Hieronder definieren we een simpele functie die een getal als input neemt en vermenigvuldigt met twee. de `time.sleep(1)` laat het process een seconde wachten voordat het weer door kan

In [ ]:
def slow_function(val):
    time.sleep(1)  # Wacht een seconde
    return val * 2

Hieronder doen we nog niets parallel, maar roepen we de functie aan in een loop en slaan we de resultaten dynamisch op in een lijst.

In [ ]:
start = time.time()

# Calculation
results = []
inputs = range(20)
for i in inputs:
    results.append(slow_function(i))
    
stop = time.time()
print('Results: {}'.format(results))
print('Elapsed time: {} seconds'.format(start - stop))

## Duurt lang!
Dat duurde lang heh? Dat kan sneller! Probeer de code hieronder maar eens

In [ ]:
start = time.time()

# Calculation
jobs = (delayed(slow_function)(i) for i in inputs)
results = Parallel(n_jobs=5)(jobs)

stop = time.time()
print('Results: {}'.format(results))
print('Elapsed time: {} seconds'.format(start - stop))

## Fijn!
Zoals te zien gaat dit aanzienlijk sneller. We gebruikten hier `5 processen` die los van elkaar draaien om het rekenwerk sneller uit te kunnen voeren. Helaas duurt het dan niet 5 keer korter om de taak uit te voeren omdat er ook tijd verloren gaat tijdens het parralleliseren.

Hieronder nog even stap voor stap hoe deze regels werken:
```python
jobs = (delayed(slow_function)(i) for i in inputs)
results = Parallel(n_jobs=5)(jobs)
```

### 1. delayed
delayed is een [decorator](https://www.datacamp.com/community/tutorials/decorators-python) (waarmee nieuwe functionaliteit aan een object kan worden toegevoegd)

In dit geval zorgt de decorator `delayed` ervoor dat de functie niet wordt uitgevoerd bij het invoeren van de argumenten. om dit te illustreren de volgende regels code:

In [ ]:
delayed_slow_function = delayed(slow_function)
delayed_slow_function

`delayed_slow_function` is nu een nieuwe functie, bijna een kopie van `slow_function` maar met een iets ander gedrag, zo zien we bijvoorbeeld dat de functie niet is uitgevoerd na het invoeren van het argument `2`:

In [ ]:
job = delayed_slow_function(2)
job

in de variabele `job` staat --als tuple met ingevulde `*args` en `**kwargs`-- nog te wachten tot deze mag worden uitgevoerd. Dat kan dan als volgt:

In [ ]:
job[0](*job[1], **job[2])

Dit laatste wordt eigenlijk automatisch gedaan in de `Parallel` functie

### 2. Jobs
Het aanmaken van meerdere delayed functies zoals hierboven is wat er gebeurd in de gehele regel:

In [ ]:
jobs = (delayed(slow_function)(i) for i in inputs)
jobs

jobs is nu een [generator](https://realpython.com/introduction-to-python-generators/) object geworden die een delayed functie iteratief aanmaakt. (Generator objecten zijn vriendelijker voor het geheugen als het gaat om heel veel jobs)

Als we een element willen opvragen uit een generator, dan kan dat met

In [ ]:
job = next(jobs)
job

of voor alle elementen:

In [ ]:
list(jobs)

merk op dat de eerste mist, omdat we die al met `next(jobs)` hebben opgevraagd. Als we nu weer dezelfde regel uitvoeren krijgen we ook niets meer terug omdat de generator `jobs` al uitgeput is

In [ ]:
list(jobs)

### 3. Parallel
de `Parallel` class kan gebruikt worden om een object te starten waarmee meerdere gelijktijdig draaiende processen worden opgestart. In principe is het vaak voldoende om alleen het argument `n_jobs` op te geven. 

In principe geldt, hoe hoger n_jobs hoe sneller de rekentijd, maar er is wel een optimum. Het opstarten van jobs heeft de nodige overhead en er is een limiet aan het rekenwerk wat processoren fysiek kunnen doorvoeren. Voor een functie die 100% rekencapaciteit vereist, heeft het dus weinig zin om meer jobs op te starten dan het aantal logische processoren in jouw computer. Bovendien is er ook een limiet aan het werkgeheugen wat gelijktijdig kan worden gebruikt. Voor functies met veel wachttijd levert het wel een aanzienlijke verbetering op.

Tot slot zijn er veel manieren om de Parallelisatie op te starten zoals bijvoorbeeld [multi-threading](https://joblib.readthedocs.io/en/latest/parallel.html#thread-based-parallelism-vs-process-based-parallelism) in plaats van parallel processen. Voor details over deze verschillen en alle verschillende opties is het goed om de [documentatie](https://joblib.readthedocs.io/en/latest/) eens te raadplegen.

In [ ]:
client = Parallel(n_jobs=5)
client

### 4. resultaten
Tot slot nog de berekening aanzetten en het verzamelen van resultaten. Dat gaat door het samenvoegen van het voorgaande

In [ ]:
results = client(jobs)

In [ ]:
results

de resultaten zijn leeg, omdat de jobs eigenlijk ook nog een lege lijst was, dus die moeten we weer opnieuw aanmaken om het te laten werken.

In [ ]:
jobs = (delayed(slow_function)(i) for i in inputs)
results = client(jobs)
results

# Voorbeeld met een ingewikkeldere functie
Tot nu toe hadden we maar een argument, maar nu een functie met meer argumenten

In [ ]:
def slow_function2(v1, v2, *args, operation='*'):
    time.sleep(1) # wacht 1 seconde
    if operation == '*':
        result = v1 * v2
    elif operation == '-':
        result = v1 - v2
    else:
        raise NotImplementedError()
    for arg in args:
        result = result + arg
    return result

In [ ]:
# input voor de functie
v1s = range(20)  # 0 --> 19
v2s = range(20, 0, -1)  # 20 --> 1
operations = ['*', '-'] * 10  # ['*', '-', '*', ..., '-']
additions = [3] * 10 + [5] * 10

# jobs aanmaken, deze keer als lijst
jobs = []
for v1, v2, operation, addition in zip(v1s,
                                       v2s,
                                       operations,
                                       additions):
    jobs.append(delayed(slow_function2)(v1, v2, addition, operation=operation))
jobs

In [ ]:
results = Parallel(n_jobs=5)(jobs)
print(results)